# 作業

實作各種不一樣的方式來調整飽和 / 對比 / 明亮

1. 改變 color space 來調整飽和度
2. 實作直方圖均衡
3. alpha/ beta 調整對比 / 明亮

In [1]:
# 載入套件
import cv2
import numpy as np

img_path = 'Data/lena.png'

# 以彩色圖片的方式載入
img = cv2.imread(img_path, cv2.IMREAD_COLOR) # 載入包含 Blue, Green, Red 三個 channel 的彩色圖片

## 改變飽和度

- 轉換成 HSV color space, 改變 s channel 的值

In [2]:
# 為了要改變飽和度，我們先把 color space 轉成 HSL 格式 (OpenCV 表示順序是 HLS)
img_hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS) # 從 BGR 轉換到 HSL
change_percentage = 0.2

# 針對飽和度的值做改變，超過界線 0~1 的都會 bound
# 在 HLS color space 減少飽和度
# astype('float32') / astype('uint8') : 對資料型別進行轉換
img_hls_down = img_hls.astype('float32')
img_hls_down[..., -1] = img_hls_down[..., -1]/255 - change_percentage # 降低飽和度
img_hls_down[img_hls_down[..., -1] < 0] = 0
img_hls_down[..., -1] = img_hls_down[..., -1]*255
img_hls_down = img_hls_down.astype('uint8')

# 在 HLS color space 增加飽和度
img_hls_up = img_hls.astype('float32')
img_hls_up[..., -1] = img_hls_up[..., -1]/255 + change_percentage # 增加飽和度
img_hls_up[img_hls_up[..., -1] > 1] = 1
img_hls_up[..., -1] = img_hls_up[..., -1]*255
img_hls_up = img_hls_up.astype('uint8')

# 轉換
img_hls_down = cv2.cvtColor(img_hls_down, cv2.COLOR_HLS2BGR) # 從 HSL轉換到BGR
img_hls_up = cv2.cvtColor(img_hls_up, cv2.COLOR_HLS2BGR) # 從 HSL轉換到BGR

# 組合圖片 + 顯示圖片
# np.hstack:水平(按列順序)方式把陣列給堆疊起來
img_hls_change = np.hstack((img, img_hls_down, img_hls_up))
while True:
    cv2.imshow('change saturation', img_hls_change)
    
    # 按下任意鍵則關閉所有視窗
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows() #   cv2.destroyAllWindows(): 關閉所有 OpenCV 的視窗
        break
    """
    cv2.waitKey
    用來等待與讀取使用者按下的按鍵，而其參數是等待時間（單位為毫秒），若設定為 0 就表示持續等待至使用者按下按鍵為止。
    """

## 直方圖均衡

1. case 1: 把彩圖拆開對每個 channel 個別做直方圖均衡再組合起來
2. case 2: 轉換 color space 到 HSV 之後對其中一個 channel 做直方圖均衡

In [3]:
# case 1
# 每個 channel 個別做直方圖均衡
equalHist_by_channel = [img[..., 0], img[..., 1], img[..., 2]]
equalHist_by_channel = [cv2.equalizeHist(i) for i in equalHist_by_channel]
# 組合經過直方圖均衡的每個 channel
img_bgr_equal =  np.stack(equalHist_by_channel, axis=-1)

# case 2 - 轉換 color space 後只對其中一個 channel 做直方圖均衡
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_hsv[..., -1] = cv2.equalizeHist(img_hsv[..., -1])
img_hsv_equal = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR) # 從 HSL轉換到BGR

# 組合圖片 + 顯示圖片
# np.hstack:水平(按列順序)方式把陣列給堆疊起來
img_bgr_equalHist = np.hstack((img, img_bgr_equal, img_hsv_equal))
while True:
    # 比較 (原圖, BGR color space 對每個 channel 做直方圖均衡, HSV color space 對明度做直方圖均衡)
    cv2.imshow('bgr equal histogram', img_bgr_equalHist)
    
    # 按下任意鍵則關閉所有視窗
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows() #   cv2.destroyAllWindows(): 關閉所有 OpenCV 的視窗
        break
    """
    cv2.waitKey
    用來等待與讀取使用者按下的按鍵，而其參數是等待時間（單位為毫秒），若設定為 0 就表示持續等待至使用者按下按鍵為止。
    """

## 調整對比 / 明亮

In [4]:
# alpha: 控制對比度 (1.0~3.0)
# beta: 控制明亮度 (0~255)
# 調整對比度 / 明亮度(img, alpha=, beta=):調整對比度 / 明亮度
add_contrast = cv2.convertScaleAbs(img, alpha=2.5, beta=0)
add_lighness = cv2.convertScaleAbs(img, alpha=1.0, beta=75)

# 組合圖片 + 顯示圖片
# np.hstack:水平(按列順序)方式把陣列給堆疊起來
img_contrast_light = np.hstack((img, add_contrast, add_lighness))
while True:
    # 比較不同程度的對比 / 明亮
    cv2.imshow('adjust contrast and brighness', img_contrast_light)
   
    # 按下任意鍵則關閉所有視窗
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows() #   cv2.destroyAllWindows(): 關閉所有 OpenCV 的視窗
        break
    """
    cv2.waitKey
    用來等待與讀取使用者按下的按鍵，而其參數是等待時間（單位為毫秒），若設定為 0 就表示持續等待至使用者按下按鍵為止。
    """